In [1]:
import pandas as pd
import numpy as np

In [2]:
plant_record = []
fault_record = []
plant_list = [1,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,21,23,24,27,28,29,30,31,32,33,34,35,36,39,40]

In [3]:
def read_file(plant_index,file_path):
    train_ori_name = file_path+"/PHM_train"+str(plant_index)+".csv"
    train_ori = pd.read_csv(train_ori_name,dtype = np.float64)
    test_ori_name = file_path+"PHM_test"+str(plant_index)+".csv"
    test_ori  = pd.read_csv(test_ori_name,dtype = np.float64)
    train_label_ori_name = file_path+"PHM_trainLabel"+str(plant_index)+".csv"
    train_label_ori = pd.read_csv( train_label_ori_name,dtype = np.float64)
    test_label_ori_name = file_path+"PHM_testLabel_viseble"+str(plant_index)+".csv"
    test_label_visible_ori = pd.read_csv(test_label_ori_name,dtype = np.float64)
    
    return train_ori,test_ori,train_label_ori,test_label_visible_ori 

In [4]:
def remove_na_dup(dataset):
    dataset = dataset.drop_duplicates()
    dataset = dataset.dropna()
    return dataset

In [5]:
def onehot_encode(train_ori,test_ori):
    train_onehot = pd.DataFrame( {"ini":train_ori["season"]})
    test_onehot = pd.DataFrame( {"ini":test_ori["season"]})

    for index in range(1,5):
        train_onehot[( "season_"+str(index) )] = (train_ori["season"] == index ).astype(int)
        test_onehot[( "season_"+str(index) )] = (test_ori["season"] == index ).astype(int)

    for index in range(1,13):
        train_onehot[( "month_"+str(index) )] = (train_ori["month"] == index ).astype(int)
        test_onehot[( "month_"+str(index) )] = (test_ori["month"] == index ).astype(int)

    for index in range(1,8):
        train_onehot[( "wday_"+str(index) )] = (train_ori["wday"] == index ).astype(int)
        test_onehot[( "wday_"+str(index) )] = (test_ori["wday"] == index ).astype(int)

    for index in range(1,25):
        train_onehot[( "hour_"+str(index) )] = (train_ori["hour"] == index ).astype(int)
        test_onehot[( "hour_"+str(index) )] = (test_ori["hour"] == index ).astype(int)
        
    train_onehot = train_onehot.drop("ini",1)
    test_onehot = test_onehot.drop("ini",1)
    return train_onehot,test_onehot

In [6]:
def standardize(dataset_train,dataset_test):
    b = (dataset_test - dataset_train.mean())/dataset_train.std()
    return b

In [7]:
def combine_addzero_feature(sensor,onehot,width):
    dataset = pd.concat([onehot,sensor],axis = 1)
    dataset.columns.values
    ##add padding
    for index in range( 13**2-dataset.shape[1] ):
        dataset[("zero_"+str(index))] = 0
    return dataset

In [8]:
def add_laged_feature(dataset,dataset_ori,section):
    train_laged = dataset[ section:(dataset.shape[0]-section)]
    train_laged.index = range(train_laged.shape[0])
    train_laged_time = dataset_ori.iloc[ section:(dataset.shape[0]-section),0 ]
    train_laged_time.index = range(train_laged_time.shape[0])


    
        ##before section
    for index in range(section):
        add_lag = dataset[index:(dataset.shape[0]-section*2+index)]
        add_lag.columns = add_lag.columns.values+"_-"+str(index)
        add_lag.index = range(add_lag.shape[0])
        train_laged = pd.concat([train_laged, add_lag],axis = 1)


        ##after section
    for index in range(section):
        add_lag = dataset[(section+index):(dataset.shape[0]-section+index)]
        add_lag.columns = add_lag.columns.values+"_+"+str(index)
        add_lag.index = range(add_lag.shape[0])
        train_laged = pd.concat([train_laged, add_lag],axis = 1)

            ##combine
    train_c = pd.concat([train_laged_time,train_laged],axis = 1)


    return train_c

In [9]:
def add_label(dataset,dataset_fault):
    ##add label
    dataset["start_label"] = 0
    for index in range( dataset_fault.shape[0] ):
        label_v = range( int(dataset_fault["startTime"][index]-4) ,int(dataset_fault["startTime"][index]+5) )    
        dataset.loc[dataset.time.isin(label_v),'start_label'] = 1

    return dataset        

In [10]:
##add test visible fault to train set
def add_visible_to_train(train_sub,test_sub):  
    add_test = test_sub[test_sub.start_label == 1]
    train_sub_add = pd.concat([train_sub,add_test],axis = 0,ignore_index = True)
    train_sub_add = train_sub_add.sort_values(["time"])
    return train_sub_add

In [11]:
##random down sampling
def random_down_sampling(train_sub_add):
    train_sub_0 = train_sub_add[train_sub_add.start_label == 0]
    train_sub_0.index = range(train_sub_0.shape[0])
    train_sub_1 = train_sub_add[train_sub_add.start_label == 1]
    train_sub_1.index = range(train_sub_1.shape[0])
    train_sub_0 = train_sub_0.iloc[np.random.choice(train_sub_0.shape[0], train_sub_1.shape[0], replace=True),:]
    train_set = pd.concat([train_sub_1,train_sub_0],axis = 0,ignore_index = True)
    train_set = train_set.sort_values(["time"])
    train_set.index = range(train_set.shape[0])
    return train_set

In [12]:
##main function
def do_preprocess(width,section,read_file_path,save_file_path) :
    count = 0
    for plant_index in plant_list:
        print("plant_index: %s  total: %s"%(count,len(plant_list)))
        count = count+1
        ##for loop
        train_ori,test_ori,train_label_ori,test_label_visible_ori = read_file(plant_index,read_file_path)
        train_ori = remove_na_dup(train_ori)
        test_ori = remove_na_dup(test_ori)
        train_sensor = train_ori.drop(["time","year","season","month","wday","hour"],1)
        test_sensor = test_ori.drop(test_ori.columns[:6],1)
        train_onehot,test_onehot = onehot_encode(train_ori,test_ori)
        test_sensor = standardize(train_sensor,test_sensor)
        train_sensor = standardize(train_sensor,train_sensor)
        train_basic = combine_addzero_feature(train_sensor,train_onehot,width)
        test_basic = combine_addzero_feature(test_sensor,test_onehot,width)
        train_combine = add_laged_feature(train_basic,train_ori,section)
        test_combine = add_laged_feature(test_basic,test_ori,section)
        #different faults
        for fault_index in train_label_ori["fault"].unique().astype(int):
            print("fault: %s"%fault_index)
            ##for loop
            train_sub = train_combine
            test_sub = test_combine
            train_sub_fault = train_label_ori[ train_label_ori["fault"] == fault_index ]
            train_sub_fault.index = range(train_sub_fault.shape[0])
            test_sub_fault = test_label_visible_ori[test_label_visible_ori["fault"] == fault_index ]
            test_sub_fault.index = range(test_sub_fault.shape[0])
            train_sub = add_label(train_sub,train_sub_fault)
            test_sub = add_label(test_sub,test_sub_fault)
            train_sub_add = add_visible_to_train(train_sub,test_sub)
            test_set =  test_sub[test_sub.start_label == 0]
            train_set = train_sub_add

            ##record plant and fault
            plant_record.append(plant_index)
            fault_record.append(fault_index)

            ##write csv
            train_set_name = save_file_path+"/trainset"+str(plant_index)+"_fault"+str(fault_index)+".csv"
            train_set.to_csv(train_set_name,index = False,sep = "," )

            test_set_name = save_file_path+"/testset"+str(plant_index)+"_fault"+str(fault_index)+".csv"
            test_set.to_csv(test_set_name,index = False,sep = "," )
            
    f_name = save_file_path + "plant_list.csv"
    f = open( f_name,"w")
    for index in range( (len(plant_record)-1) ):
        f.write(str(plant_record[index])+",")
    f.write(str(plant_record[-1]))
    f.close()
    
    f_name = save_file_path + "fault_list.csv"
    f = open(f_name,"w")
    for index in range( (len(fault_record)-1) ):
        f.write(str(fault_record[index])+",")
    f.write(str(fault_record[-1]))
    f.close()


In [13]:
##parameters setting
width = 13
section = 4
read_file_path = "/media/joeytu/82F6C88FF6C8853F/PHM2017/PHM_rawdata/"
save_file_path = "/media/joeytu/82F6C88FF6C8853F/PHM2017/PHM_nosample/file/start"

##do_preprocess
do_preprocess(width,section,read_file_path,save_file_path) 

plant_index: 0  total: 31
fault: 1
fault: 2
fault: 3
fault: 4
fault: 5
plant_index: 1  total: 31
fault: 1
fault: 4
fault: 2
fault: 5
plant_index: 2  total: 31
fault: 1
fault: 2
fault: 4
fault: 3
fault: 5
plant_index: 3  total: 31
fault: 1
fault: 4
fault: 2
fault: 3
plant_index: 4  total: 31
fault: 1
fault: 2
fault: 3
fault: 4
plant_index: 5  total: 31
fault: 4
fault: 3
fault: 2
fault: 1
fault: 5
plant_index: 6  total: 31
fault: 2
fault: 1
fault: 5
fault: 4
fault: 3
plant_index: 7  total: 31
fault: 1
fault: 2
fault: 5
plant_index: 8  total: 31
fault: 1
fault: 5
fault: 4
fault: 3
plant_index: 9  total: 31
fault: 1
fault: 4
plant_index: 10  total: 31
fault: 1
fault: 3
fault: 2
fault: 4
plant_index: 11  total: 31
fault: 4
fault: 1
fault: 2
fault: 5
fault: 3
plant_index: 12  total: 31
fault: 2
fault: 1
fault: 4
fault: 5
fault: 3
plant_index: 13  total: 31
fault: 2
fault: 1
fault: 4
fault: 3
fault: 5
plant_index: 14  total: 31
fault: 2
fault: 1
fault: 5
fault: 4
fault: 3
plant_index: 15  tot